In [1]:
!pip install comet_ml --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.4/450.4 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.2/502.2 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import comet_ml
comet_ml.init(project_name='tirendaz-optimizer')

Please enter your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Comet API key is valid
COMET WARNING: running in Google Colab, but can't find mounted drive. Using HOME instead
COMET WARNING: if drive is mounted, set COMET_CONFIG to save config there
COMET INFO: Comet API key saved in /root/.comet.config


In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical

def build_model_graph(experiment):
    model = Sequential()
    model.add(
        Dense(
            experiment.get_parameter("first_layer_units"),
            activation="sigmoid",
            input_shape=(784,),
        )
    )
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(128, activation="sigmoid"))
    model.add(Dense(10, activation="softmax"))
    model.compile(
        loss="categorical_crossentropy",
        optimizer=RMSprop(),
        metrics=["accuracy"],
    )
    return model

def train(experiment, model, x_train, y_train, x_test, y_test):
    model.fit(
        x_train,
        y_train,
        batch_size=experiment.get_parameter("batch_size"),
        epochs=experiment.get_parameter("epochs"),
        validation_data=(x_test, y_test),
    )

def evaluate(experiment, model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Score %s", score)

def get_dataset():
    num_classes = 10

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)
    x_train = x_train.astype("float32")
    x_test = x_test.astype("float32")
    x_train /= 255
    x_test /= 255
    print(x_train.shape[0], "train samples")
    print(x_test.shape[0], "test samples")

    # convert class vectors to binary class matrices
    y_train = to_categorical(y_train, num_classes)
    y_test = to_categorical(y_test, num_classes)

    return x_train, y_train, x_test, y_test


# Get the dataset:
x_train, y_train, x_test, y_test = get_dataset()



11490434/11490434 [==============================] - 0s 0us/step
60000 train samples
10000 test samples


In [4]:
# The optimization config:
config = {
    "algorithm": "bayes",
    "name": "Optimize MNIST Network",
    "spec": {"maxCombo": 5, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {
            "type": "integer",
            "mu": 500,
            "sigma": 50,
            "scalingType": "normal",
        },
        "batch_size": {"type": "discrete", "values": [64, 128, 256]},
    },
    "trials": 1,
}

opt = comet_ml.Optimizer(config)

COMET INFO: COMET_OPTIMIZER_ID=b2c166cd48884520a8e65ca4b1a1bbc0
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 'infinite', 'endTime': None, 'id': 'b2c166cd48884520a8e65ca4b1a1bbc0', 'lastUpdateTime': None, 'maxCombo': 5, 'name': 'Optimize MNIST Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [64, 128, 256]}, 'first_layer_units': {'mu': 500, 'scalingType': 'normal', 'sigma': 50, 'type': 'integer'}}, 'predictor': None, 'spec': {'gridSize': 10, 'maxCombo': 5, 'metric': 'loss', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 17147797832, 'state': {'mode': None, 'seed': None, 'sequence': [], 'sequence_i': 0, 'sequence_pid': None, 'sequence_retry': 0, 'sequence_retry_count': 0}, 'status': 'running', 'suggestion_count': 0, 'trials': 1, 'version': '2.0.17'}


In [5]:
for experiment in opt.get_experiments():
    # Log parameters, or others:
    experiment.log_parameter("epochs", 10)

    # Build the model:
    model = build_model_graph(experiment)

    # Train it:
    train(experiment, model, x_train, y_train, x_test, y_test)

    # How well did it do?
    evaluate(experiment, model, x_test, y_test)

    # Optionally, end the experiment:
    experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Failed to calculate active processors count. Fall back to default CPU count 1
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.com https://www.comet.com/tirendaz/tirendaz-optimizer/bdfe378d35344c228f24090a6047ddc1

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/10
469/469 [==============================] - 5s 5ms/step - loss: 0.6235 - accuracy: 0.8291 - val_loss: 0.2610 - val_accuracy: 0.9225
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.2296 - accuracy: 0.9314 - val_loss: 0.1927 - val_accuracy: 0.9416
Epoch 3/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1645 - accuracy: 0.9504 - val_loss: 0.1423 - val_accuracy: 0.9558
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.1260 - accuracy: 0.9621 - val_loss: 0.1287 - val_accuracy: 0.9612
Epoch 5/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1009 - accuracy: 0.9700 - val_loss: 0.1046 - val_accuracy: 0.9687
Epoch 6/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0842 - accuracy: 0.9742 - val_loss: 0.0926 - val_accuracy: 0.9707
Epoch 7/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0710 - accuracy: 0.9782 - val_loss: 0.0835 - val_accuracy: 0.9739
Epoch 

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/tirendaz/tirendaz-optimizer/bdfe378d35344c228f24090a6047ddc1
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.8290833234786987, 0.9865166544914246)
COMET INFO:     batch_accuracy [470]         : (0.14204545319080353, 1.0)
COMET INFO:     batch_loss [470]             : (0.011746536940336227, 2.366264820098877)
COMET INFO:     epoch_duration [10]          : (2.055164474000037, 5.049704657999996)
COMET INFO:     loss [10]                    : (0.04483644291758537, 0.6234879493713379)
COMET INFO:     val_accuracy [10]            : (0.9225000143051147, 0.9772999882698059)
COMET INFO:     val_loss [10]                : (0.0746418908238411, 0.26103517413139343)
COMET INFO:     validate_batch_accuracy [80] : (0.

Epoch 1/10
235/235 [==============================] - 2s 5ms/step - loss: 0.8638 - accuracy: 0.7666 - val_loss: 0.3592 - val_accuracy: 0.8963
Epoch 2/10
235/235 [==============================] - 1s 5ms/step - loss: 0.2911 - accuracy: 0.9150 - val_loss: 0.2400 - val_accuracy: 0.9283
Epoch 3/10
235/235 [==============================] - 1s 5ms/step - loss: 0.2170 - accuracy: 0.9354 - val_loss: 0.2036 - val_accuracy: 0.9353
Epoch 4/10
235/235 [==============================] - 1s 5ms/step - loss: 0.1735 - accuracy: 0.9475 - val_loss: 0.1645 - val_accuracy: 0.9503
Epoch 5/10
235/235 [==============================] - 1s 5ms/step - loss: 0.1419 - accuracy: 0.9579 - val_loss: 0.1340 - val_accuracy: 0.9571
Epoch 6/10
235/235 [==============================] - 1s 5ms/step - loss: 0.1199 - accuracy: 0.9642 - val_loss: 0.1316 - val_accuracy: 0.9602
Epoch 7/10
235/235 [==============================] - 1s 5ms/step - loss: 0.1029 - accuracy: 0.9692 - val_loss: 0.1246 - val_accuracy: 0.9627
Epoch 

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/tirendaz/tirendaz-optimizer/a93c7293c14946d09f9b726268edd965
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.7665500044822693, 0.9796333312988281)
COMET INFO:     batch_accuracy [240]         : (0.09765625, 0.98828125)
COMET INFO:     batch_loss [240]             : (0.03922456130385399, 2.4998950958251953)
COMET INFO:     epoch_duration [10]          : (1.0822716289999903, 1.776850546999981)
COMET INFO:     loss [10]                    : (0.06699348241090775, 0.8637544512748718)
COMET INFO:     val_accuracy [10]            : (0.8963000178337097, 0.9710000157356262)
COMET INFO:     val_loss [10]                : (0.09790860116481781, 0.3592005968093872)
COMET INFO:     validate_batch_accuracy [40] : (0.8

...

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/tirendaz/tirendaz-optimizer/abc861ec7f3f45828478061f92866fa4
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.8317333459854126, 0.9865333437919617)
COMET INFO:     batch_accuracy [470]         : (0.109375, 1.0)
COMET INFO:     batch_loss [470]             : (0.024615202099084854, 2.3786988258361816)
COMET INFO:     epoch_duration [10]          : (2.0774499180000134, 2.882814673999974)
COMET INFO:     loss [10]                    : (0.042473893612623215, 0.6268631219863892)
COMET INFO:     val_accuracy [10]            : (0.9190999865531921, 0.9753000140190125)
COMET INFO:     val_loss [10]                : (0.08114247769117355, 0.27101123332977295)
COMET INFO:     validate_batch_accuracy [80] : (0.8932291

Epoch 1/10
938/938 [==============================] - 5s 4ms/step - loss: 0.4958 - accuracy: 0.8572 - val_loss: 0.2389 - val_accuracy: 0.9274
Epoch 2/10
938/938 [==============================] - 4s 4ms/step - loss: 0.1902 - accuracy: 0.9427 - val_loss: 0.1584 - val_accuracy: 0.9511
Epoch 3/10
938/938 [==============================] - 4s 4ms/step - loss: 0.1307 - accuracy: 0.9606 - val_loss: 0.1194 - val_accuracy: 0.9629
Epoch 4/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0999 - accuracy: 0.9702 - val_loss: 0.0956 - val_accuracy: 0.9705
Epoch 5/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0797 - accuracy: 0.9761 - val_loss: 0.0906 - val_accuracy: 0.9725
Epoch 6/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0656 - accuracy: 0.9802 - val_loss: 0.0937 - val_accuracy: 0.9717
Epoch 7/10
938/938 [==============================] - 4s 4ms/step - loss: 0.0536 - accuracy: 0.9841 - val_loss: 0.0790 - val_accuracy: 0.9754
Epoch 

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/tirendaz/tirendaz-optimizer/b3bb1279b93a4e2aaef3b8579eabd902
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                 : (0.8572333455085754, 0.9900166392326355)
COMET INFO:     batch_accuracy [940]          : (0.078125, 1.0)
COMET INFO:     batch_loss [940]              : (0.015943821519613266, 2.3394839763641357)
COMET INFO:     epoch_duration [10]           : (3.9773497120000343, 4.693525331999979)
COMET INFO:     loss [10]                     : (0.03221554681658745, 0.4957592189311981)
COMET INFO:     val_accuracy [10]             : (0.9273999929428101, 0.9761999845504761)
COMET INFO:     val_loss [10]                 : (0.07897008955478668, 0.23892709612846375)
COMET INFO:     validate_batch_accuracy [160] : (0.

Epoch 1/10
235/235 [==============================] - 2s 5ms/step - loss: 0.8585 - accuracy: 0.7705 - val_loss: 0.3860 - val_accuracy: 0.8876
Epoch 2/10
235/235 [==============================] - 1s 5ms/step - loss: 0.2930 - accuracy: 0.9129 - val_loss: 0.2372 - val_accuracy: 0.9293
Epoch 3/10
235/235 [==============================] - 1s 5ms/step - loss: 0.2161 - accuracy: 0.9345 - val_loss: 0.1943 - val_accuracy: 0.9404
Epoch 4/10
235/235 [==============================] - 1s 5ms/step - loss: 0.1697 - accuracy: 0.9492 - val_loss: 0.1581 - val_accuracy: 0.9504
Epoch 5/10
235/235 [==============================] - 1s 5ms/step - loss: 0.1381 - accuracy: 0.9587 - val_loss: 0.1288 - val_accuracy: 0.9622
Epoch 6/10
235/235 [==============================] - 1s 5ms/step - loss: 0.1169 - accuracy: 0.9652 - val_loss: 0.1243 - val_accuracy: 0.9610
Epoch 7/10
235/235 [==============================] - 1s 5ms/step - loss: 0.0997 - accuracy: 0.9699 - val_loss: 0.1231 - val_accuracy: 0.9635
Epoch 

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/tirendaz/tirendaz-optimizer/ac3720b78d5d4f4e9af330ff7f0610aa
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [10]                : (0.7705166935920715, 0.9799500107765198)
COMET INFO:     batch_accuracy [240]         : (0.125, 0.9838709831237793)
COMET INFO:     batch_loss [240]             : (0.055389631539583206, 2.6230287551879883)
COMET INFO:     epoch_duration [10]          : (1.1169249739999714, 1.837258841999983)
COMET INFO:     loss [10]                    : (0.06513877958059311, 0.8585355281829834)
COMET INFO:     val_accuracy [10]            : (0.8876000046730042, 0.9722999930381775)
COMET INFO:     val_loss [10]                : (0.09269298613071442, 0.38603508472442627)
COMET INFO:     validate_batch_accuracy [40] :